
#QCI
####author: Jan T. (IR) en Team D&A

In [1]:
###Load packages
import pandas       as pd   #required for data wrangling
import pingouin     as pg   #required for Cronbach's Alpha

In [2]:
### Load data
# Use sample data because of unavailability of QCI data
qcidata = pd.DataFrame({   'Vraag3': [3, 2, 2, 3, 2, 2, 3, 3, 2, 1],
                            'Vraag4': [1, 1, 1, 2, 3, 3, 2, 3, 3, 3],
                            'Vraag5': [1, 1, 2, 1, 2, 3, 3, 3, 2, 3]})

##Interne consistentie
Om interne consistentie per subschaal te meten is [Cronbach's Alpha](https://www.scribbr.nl/statistiek/cronbachs-alpha/) gebruikt, deze is geïnterpreteerd met gebruik van de volgende [bron](https://towardsdatascience.com/cronbachs-alpha-theory-and-application-in-python-d2915dd63586).

In [3]:
alpha = pg.cronbach_alpha(data=qcidata[['Vraag3', 'Vraag4', 'Vraag5']])

(0.2544642857142858, array([-1.184,  0.799]))